# Pytorch_lightning_audio

In [5]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from torchvision import transforms

In [6]:
from tqdm.notebook import tqdm 
import numpy as np

# Datasets読み込み

In [7]:
datasets_dir = "/disk107/yamamoto/datasets.npz"
datasets = np.load(datasets_dir)
data = datasets['arr_0']
label = datasets['arr_1']

In [8]:
practice_d = data
practice_l = label
print(practice_d[1,:,:].shape)

# 1次元配列変換
_d = np.array([practice_d[i].reshape(-1,) for i in tqdm(range(len(practice_d)))])
_l = np.array([practice_l[i].reshape(-1,) for i in tqdm(range(len(practice_l)))])

transform = transforms.Compose([
    transforms.ToTensor(), 
#     transforms.Lambda(lambda x: x.view(-1))
])

class MyDataset(torch.utils.data.Dataset):

    def __init__(self, data, label, transform=None):
        self.transform = transform
        self.data = data
        self.data_num = len(data)
        self.label = label

    def __len__(self):
        return self.data_num

    def __getitem__(self, idx):
        if self.transform:
          out_data = self.transform(self.data)[0][idx]
          out_label = self.label[idx]
        else:
          out_data = self.data[idx]
          out_label =  self.label[idx]

        return out_data, out_label

(127, 512)


In [9]:
mydataset = MyDataset(_d, _l, transform)

# ネットワークの定義
**sizeの設定気をつけて！**

In [10]:
input_size = 127*512
hidden_size = 512*512
output_size = 127*512

## 1 DNN

In [3]:
class LitAutoEncoder(pl.LightningModule):
    def __init__(self):# ここで計算を定義します
        super(LitAutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
#             nn.Linear(hidden_size,hidden_size),
#             nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,output_size),
            nn.Sigmoid()
        )

#         self.batch_size = batch_size
    
    def forward(self, x):# ここは推論フェーズにのみ使われます
        # Lightningではforwardは予測/推論のアクションを定義します。
        embedding = self.encoder(x)
        return embedding
    
    def configure_optimizers(self):# Optimaizerの設定
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx):# 完全なトレーニングループ
        # training_stepでは訓練ループを定義していて、これはforwardとは独立しています
        x, y = batch
#         x = x.view(x.size(0), -1)
        z = self.encoder(x)  # forwardを呼び出す self(x) でもよい
        loss = F.mse_loss(z, x)
        self.log("train_loss", loss)
        return loss
    # lossを返している限りはLightningによって自動で最適化される（backward、optimizerの更新）

    def validation_step(self, val_batch, batch_idx):#完全な検証ループ
        # 検証ステップ
        x, y = val_batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        loss = F.mse_loss(z, x)
        self.log("val_loss", loss)
        return loss

NameError: name 'pl' is not defined

In [81]:
# train : val = 80% : 20%
n_train = int(len(mydataset) * 0.8)
n_val = len(mydataset) - n_train
train, val = random_split(mydataset, [n_train, n_val])

print(len(train),"train, ",len(val),"val")

12480 train,  3120 val


In [24]:
# train : val = 80% : 20%
n_train = int(len(mydataset) * 0.8)
n_val = len(mydataset) - n_train
train, val = random_split(mydataset, [n_train, n_val])

#モデルの呼び出しと初期化
autoencoder = LitAutoEncoder()# 学習モデルのインスタンス化
#トレーニング開始
trainer = pl.Trainer(max_epochs=100, gpus=4)# 学習用のインスタンス化と学習の設定

trainer.fit(autoencoder, 
            DataLoader(train, batch_size = 10), 
            DataLoader(val, batch_size = 10))# 学習ループ実行
# batchサイズはdata数に対して割り切れる数を渡す
# ここでdataLoaderした訓練データと検証データをわたす。

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


RuntimeError: CUDA out of memory. Tried to allocate 4.21 GiB (GPU 0; 8.00 GiB total capacity; 4.21 GiB already allocated; 1.96 GiB free; 4.22 GiB reserved in total by PyTorch)